In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
DF = pd.read_csv('grade_sim.csv', sep=';', encoding='latin-1', low_memory=False)

In [3]:
DF.drop(['PERIODO', 'DESC_MATERIAL' , 'DEPARTAMENTO', 'CD', 'ESTOQUE_VALOR', 'CMV',
         'PREV_DIA', 'GRADE', 'EST_SIM_VL', 'VDA_SIM_CMV'], axis=1, inplace=True)

In [4]:
DIAS = DF['DIA'].unique()

In [5]:
dict_vda = {name : group.sum() 
            for name, group in DF['VENDA_QTDE'].groupby([DF['ITEM'],DF['LOJA'], DF['DIA']])}

In [6]:
dict_est = {name : group.sum() 
            for name, group in DF['EST_SIM_QTDE'].groupby([DF['ITEM'],DF['LOJA'], DF['DIA']])}

In [7]:
dict_abs = {name : 0 
            for name, group in DF['ABAST'].groupby([DF['ITEM'],DF['LOJA'], DF['DIA']])}

In [8]:
dc = {name: group.values[0] for name, group in DF['LEAD_TIME'].groupby([DF['ITEM'],DF['LOJA']])}
dc_aux = {name: group.values[0] for name, group in DF['LEAD_TIME'].groupby([DF['ITEM'],DF['LOJA']])}

In [9]:
dc_dia = {name: group.values[0] for name, group in DF['LEAD_TIME'].groupby([DF['ITEM'],DF['LOJA'],DF['DIA']])}
dc_aux_dia = {name: group.values[0] for name, group in DF['LEAD_TIME'].groupby([DF['ITEM'],DF['LOJA'],DF['DIA']])}

In [10]:
dict_key = {name: False for name, group in DF['CHAVE'].groupby([DF['ITEM'],DF['LOJA']])}

In [11]:
dict_grade = {name: group.values[0] for name, group in DF['GRADE_M'].groupby([DF['ITEM'],DF['LOJA'], DF['DIA']])}

In [12]:
DF['DICT_KEY_DAY'] = list(zip(DF['ITEM'],DF['LOJA'], DF['DIA']))

In [13]:
DF['DICT_STOCK'] = list(zip(DF['ITEM'],DF['LOJA'], DF['DIA']-1))

In [14]:
DF['DICT_KEY'] = list(zip(DF['ITEM'],DF['LOJA']))

In [15]:
DF['DICT_ABAST'] = list(zip(DF['ITEM'], DF['LOJA'], DF['DIA']-DF['LEAD_TIME']+1))

In [16]:
#PRIMEIRO DIA (FIRST DAY)
counter = 0
inicio = time.time()        
for dia in DIAS[:1]:
    DIA_ATUAL = dia
    
    #VENDA DO DIA
    DF['VDA_SIM_QTDE'] = np.where(DF['DIA'] == DIA_ATUAL, 
                                  DF['DICT_KEY_DAY'].map(dict_vda), 
                                  DF['VDA_SIM_QTDE'])
                                 
    #ABASTECIMENTO
    DF['ABAST'] = np.where(DF['COUNTER'] == 1, 
                           DF['DICT_ABAST'].map(dict_abs), 
                           0)
                          
    #ESTOQUE AO FIM DO DIA
    DF['EST_SIM_QTDE'] = np.where(DF['DIA'] == DIA_ATUAL,
                                  ((DF['DICT_KEY_DAY'].map(dict_est)) - DF['VDA_SIM_QTDE'] + DF['ABAST']), 
                                  DF['EST_SIM_QTDE'])    
    
    #VERIFICANDO FALTA
    DF['VDA_SIM_QTDE'] = np.where((DF['EST_SIM_QTDE'] <= 0)&(DF['DIA'] == DIA_ATUAL), 
                                  0, 
                                  DF['VDA_SIM_QTDE'])
    
    DF['EST_SIM_QTDE'] = np.where((DF['EST_SIM_QTDE'] <= 0)&(DF['DIA'] == DIA_ATUAL), 
                                  DF['DICT_KEY_DAY'].map(dict_est), 
                                  DF['EST_SIM_QTDE'])
    
    
    #ATUALIZANDO DICIONÁRIO DE POSIÇÃO DE ESTOQUE
    dict_est = {name : group.sum() 
                for name, group in DF['EST_SIM_QTDE'].groupby([DF['ITEM'],DF['LOJA'], DF['DIA']])}

        
    #DISPARADOR DE PEDIDO
    DF['CHAVE'] = np.where((DF['EST_SIM_QTDE'] <= (DF['GRADE_M']*0.85))&(DF['DIA'] == DIA_ATUAL), 
                           True, 
                           DF['CHAVE'])
        
    dict_key_aux = {str(name): group.values[0] 
                    for name, group in DF['CHAVE'].groupby([DF['ITEM'],DF['LOJA'], DF['DIA']])}
    dict_key = {name: dict_key_aux[str(name)[:-1]+', '+str(DIA_ATUAL)+')'] for name in dict_key}
    
    #CONTADOR DO PEDIDO
    DF['COUNTER'] = np.where((DF['CHAVE'] == True)&(DF['DIA'] == DIA_ATUAL), 
                             DF['DICT_KEY'].map(dc), 
                             DF['COUNTER'])
    
        
    #ATUALIZANDO CONTADORES
    dc = {name: dc[name] - 1 if dict_key[name] == True else dc[name] 
          for name in dc}
    dc = {name: dc_aux[name] if dc[name] == 0 else dc[name] 
          for name in dc}
    
    dc_dia = {name: group.values[0] for name, group in DF['COUNTER'].groupby([DF['ITEM'],DF['LOJA'],DF['DIA']])}
        
    #ATUALIZANDO ABASTECIMENTO 
    dict_abs ={name: dict_grade[name] - dict_est[name] 
               if (dc_dia[name] == dc_aux_dia[name])&(dict_key_aux[str(name)] == True) 
               else dict_abs[name] 
               for name in dict_abs}    
    
    final = time.time()
    counter += 1
    final = time.time()
    print('Progress: {}%'.format(round((counter / len(DIAS[:1])) * 100, 2)) +
          ' Tempo de Processamenteo (Parcial): ' + str(final - inicio), end="\r", flush=True)

In [17]:
#DEMAIS DIAS (OTHER DAYS)
counter = 0
inicio = time.time()        
for dia in DIAS[1:]:
    DIA_ATUAL = dia
    
    #VENDA DO DIA
    DF['VDA_SIM_QTDE'] = np.where(DF['DIA'] == DIA_ATUAL, 
                                  DF['DICT_KEY_DAY'].map(dict_vda), 
                                  DF['VDA_SIM_QTDE'])
                                 
    #ABASTECIMENTO
    DF['ABAST'] = np.where(DF['COUNTER'] == 1, 
                           DF['DICT_ABAST'].map(dict_abs), 
                           0)
                          
    #ESTOQUE AO FIM DO DIA
    DF['EST_SIM_QTDE'] = np.where(DF['DIA'] == DIA_ATUAL, 
                                  ((DF['DICT_STOCK'].map(dict_est)) - DF['VDA_SIM_QTDE'] + DF['ABAST']), 
                                  DF['EST_SIM_QTDE'])    
    
    #VERIFICANDO FALTA
    DF['VDA_SIM_QTDE'] = np.where((DF['EST_SIM_QTDE'] <= 0)&(DF['DIA'] == DIA_ATUAL), 
                                  0, 
                                  DF['VDA_SIM_QTDE'])
    
    DF['EST_SIM_QTDE'] = np.where((DF['EST_SIM_QTDE'] <= 0)&(DF['DIA'] == DIA_ATUAL), 
                                  DF['DICT_STOCK'].map(dict_est), 
                                  DF['EST_SIM_QTDE'])
    
    
    #ATUALIZANDO DICIONÁRIO DE POSIÇÃO DE ESTOQUE
    dict_est = {name : group.sum() 
                for name, group in DF['EST_SIM_QTDE'].groupby([DF['ITEM'],DF['LOJA'], DF['DIA']])}

        
    #DISPARADOR DE PEDIDO
    DF['CHAVE'] = np.where((DF['EST_SIM_QTDE'] <= (DF['GRADE_M']*0.85))&(DF['DIA'] == DIA_ATUAL), 
                           True, 
                           DF['CHAVE'])
        
    dict_key_aux = {str(name): group.values[0] 
                    for name, group in DF['CHAVE'].groupby([DF['ITEM'],DF['LOJA'], DF['DIA']])}
    dict_key = {name: dict_key_aux[str(name)[:-1]+', '+str(DIA_ATUAL)+')'] for name in dict_key}
    
    #CONTADOR DO PEDIDO
    DF['COUNTER'] = np.where((DF['CHAVE'] == True)&(DF['DIA'] == DIA_ATUAL), 
                             DF['DICT_KEY'].map(dc), 
                             DF['COUNTER'])
    
        
    #ATUALIZANDO CONTADORES
    dc = {name: dc[name] - 1 if dict_key[name] == True else dc[name] 
          for name in dc}
    dc = {name: dc_aux[name] if dc[name] == 0 else dc[name] 
          for name in dc}
    
    dc_dia = {name: group.values[0] for name, group in DF['COUNTER'].groupby([DF['ITEM'],DF['LOJA'],DF['DIA']])}
        
    #ATUALIZANDO ABASTECIMENTO 
    dict_abs ={name: dict_grade[name] - dict_est[name] 
               if (dc_dia[name] == dc_aux_dia[name])&(dict_key_aux[str(name)] == True) 
               else dict_abs[name] 
               for name in dict_abs}    
    
    final = time.time()
    counter += 1
    final = time.time()
    print('Progress: {}%'.format(round((counter / len(DIAS[1:])) * 100, 2)) +
          ' Tempo de Processamenteo (Parcial): ' + str(final - inicio), end="\r", flush=True)

In [18]:
DF.drop(['DICT_KEY_DAY','DICT_STOCK','DICT_KEY','DICT_ABAST'], axis=1, inplace=True)

In [19]:
DF

,ITEM,LOJA,ESTOQUE_QTDE,VENDA_QTDE,DIA,PREV_DIA_M,LEAD_TIME,GRADE_M,ESTOQUE_ALVO,EST_SIM_QTDE,VDA_SIM_QTDE,CHAVE,ABAST,COUNTER
0,4058855,1000,173,0,1,11,5,55,40.0,55.0,0,False,0.0,0
1,4058855,1000,172,1,2,11,5,55,45.0,54.0,1,False,0.0,0
2,4058855,1000,163,9,3,11,5,55,44.0,45.0,9,True,0.0,5
3,4058855,1000,149,14,4,11,5,55,38.0,31.0,14,True,0.0,4
4,4058855,1000,142,7,5,11,5,55,31.0,24.0,7,True,0.0,3
5,4058855,1000,133,9,6,11,5,55,22.0,15.0,9,True,0.0,2
6,4058855,1000,127,6,7,11,5,55,23.0,9.0,6,True,10.0,1
7,4058855,1000,119,8,8,11,5,55,22.0,1.0,8,True,0.0,5
8,4058855,1000,111,8,9,11,5,55,18.0,1.0,0,True,0.0,4
9,4058855,1000,171,0,10,11,5,55,27.0,1.0,0,True,0.0,3
